Queste funzioni fanno uso della variabile u che
corrisponde all'unita’ di arrotondamento.

In [50]:
import sys
u = sys.float_info.epsilon
real_min = sys.float_info.min

In [85]:
import math

def bisez(fn, a, b, tol):
    if a<0 and b>0 and fn(0) == 0:
        return 0
    while abs(b-a) > 2*u*(min(abs(a), abs(b)))+tol:
        m = (a+b) / 2
        fm = fn(m)
        if fn(a)*fm < 0:
            b=m
        else:
            a=m
    return (a+b)/2

def false_position(fn, a, b, tol):
    prev_y = None
    fa = fn(a)
    fb = fn(b)
    while True:
        y = b - (fb * (b - a)) / (fb - fa)
        if prev_y is not None and abs(y-prev_y) < 2*u*(min(abs(y),abs(prev_y)))+tol:
            return y
        fy = fn(y)
        if fa*fy < 0:
            b=y
            fb=fy
        else:
            a=y
            fa=fy
        prev_y = y
        
def newton(fn, fn_p, x0, tol):
    n=0
    max_iter=50
    small_real = 100*real_min
    
    x_next = x0
    x_curr = x0 + 1
    while abs(x_next - x_curr) > 2*u*min(abs(x_next), abs(x_curr))+tol and n < max_iter:
        x_curr = x_next
        fx_curr = fn(x_curr)
        if abs(fx_curr)>=small_real:
            x_next = x_curr - (fx_curr / fn_p(x_curr))
            n=n+1
    return x_next

def secant(fn, x0, x1, tol):
    n=0
    max_iter=50
    small_real=100*real_min
    
    x_next = x1
    x_curr = x0
    fx_curr = fn(x0)
    while abs(x_next - x_curr) > 2*u*min(abs(x_next), abs(x_curr)) and n < max_iter:
        x_prev = x_curr
        fx_prev = fx_curr
        x_curr = x_next
        fx_curr = fn(x_curr)
        if (abs(fx_curr)>=small_real):
            x_next = x_curr - fx_curr * (x_curr-x_prev) / (fx_curr-fx_prev)
            n=n+1
    return x_next

In [87]:
import unittest

class TestRoots(unittest.TestCase):
    def assert_root(self, fn, approx_method, a, b, tol=1e-10):
        root = fn(approx_method(fn, a, b, tol))
        self.assertAlmostEqual(root, 0, delta=tol*10)
    
    def assert_secant_root(self, fn, x0, x1, tol):
        root = fn(secant(fn, x0, x1, tol))
        self.assertAlmostEqual(root, 0, delta=tol*10)
    
    def test_bisez(self):
        self.assert_root(lambda x: (4*x-7)/(x-2), bisez, 1, 1.9)
        self.assert_root(lambda x: math.sin(x) - x, bisez, -1, 1)
    
    def test_false_position(self):
        self.assert_root(lambda x: x**3 - x - 2, false_position, 1, 2)
        self.assert_root(lambda x: math.exp(x) - 3, false_position, 0, 2)
        
    def test_newton(self):
        fn = lambda x: x**3 -2*x + 2
        fn_p = lambda x: 3*x**2 -2
        x0 = -2
        tol = 1e-6
        root = fn(newton(fn, fn_p, x0, tol))
        self.assertAlmostEqual(root, 0, delta=tol*10)
        
    def test_secant(self):
        self.assert_secant_root(lambda x: x**3 - 2*x + 2, -2, 1, 1e-6)
        self.assert_secant_root(lambda x: x**2 - 4, 0, 3, 1e-6)
        self.assert_secant_root(lambda x: math.exp(x) - 3, 0, 2, 1e-6)

        
def run_tests(test_class):
    suite = unittest.TestLoader().loadTestsFromTestCase(test_class)
    unittest.TextTestRunner(verbosity=2).run(suite)
    
run_tests(TestRoots)

test_bisez (__main__.TestRoots.test_bisez) ... ok
test_false_position (__main__.TestRoots.test_false_position) ... ok
test_newton (__main__.TestRoots.test_newton) ... ok
test_secant (__main__.TestRoots.test_secant) ... ok

----------------------------------------------------------------------
Ran 4 tests in 0.004s

OK


In [71]:
fn = lambda x: 2*3*(x - 3) + (3)**2 - 4
x_star = bisez(fn, 0, 3, 1e-5)
x_star

2.1666669845581055